## Combine all subject data

In [1]:
import pandas as pd
import glob

In [2]:
all_file = glob.glob('../data/processed/apple_watch_format/S*_apple_watch.csv')

In [3]:
df_all = pd.concat([pd.read_csv(file) for file in all_file], ignore_index=True, axis=0)
df_all

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,HR,IBI,HRV,label,subject
0,-46.582292,-6.051042,32.276042,64.614681,4.654549,69.224135,1152.500000,619.704125,0,4
1,-43.976042,-31.740625,-19.667708,64.940738,4.375065,79.507543,915.322581,738.055935,0,4
2,-56.694792,-7.480208,-23.934375,65.363949,1.140627,72.979079,1045.138889,753.696819,0,4
3,-33.206250,-8.705208,-54.866667,64.771843,0.621698,78.275497,1008.620690,855.633087,0,4
4,-33.027083,-8.196875,-54.929167,64.693208,0.778812,68.677229,1160.156250,854.699485,0,4
...,...,...,...,...,...,...,...,...,...,...
1145,-19.776042,-15.045833,38.236458,63.668923,4.141692,88.756221,754.807692,334.043583,0,11
1146,-6.693750,-31.478125,44.279167,62.952634,2.308099,92.192958,756.410256,424.894772,0,11
1147,17.828125,-51.720833,28.890625,62.335270,0.898838,80.435572,858.901515,471.185081,0,11
1148,6.482292,-54.142708,29.834375,62.491297,1.131574,84.352701,820.312500,472.101117,0,11


In [19]:
df_all['label'].value_counts()

label
0    803
1    347
Name: count, dtype: int64

In [20]:
803-347

456

In [ ]:
# เอาข้อมูลของ focus อย่างเดียว เพื่อเอาไปทำ CTGan
df_focus = df_all[df_all['label'] == 1].drop(columns=["label"])
df_focus

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,HR,IBI,HRV,subject
53,-33.983333,50.954167,3.877083,65.608231,6.047003,78.716987,922.851562,786.794839,4
54,-43.152083,45.087500,6.021875,65.155219,3.388475,78.829055,973.060345,882.758176,4
55,-31.486458,55.301042,3.812500,64.994702,2.890565,75.595658,995.689655,918.976223,4
56,-25.517708,59.012500,4.262500,65.296136,6.760261,76.516211,979.166667,884.721245,4
57,-21.538542,60.078125,7.558333,64.805785,2.594831,76.616381,1016.163793,989.886780,4
...,...,...,...,...,...,...,...,...,...
1131,-54.560417,6.232292,-35.521875,65.491437,1.554834,60.874479,1260.869565,820.167060,11
1132,-58.415625,1.589583,-29.419792,65.502716,1.634410,81.599224,936.491935,818.427895,11
1133,-60.750000,-0.440625,-24.409375,65.505751,0.943566,74.253981,1011.160714,680.998297,11
1134,-61.571875,-1.518750,-22.296875,65.512675,0.724452,82.265634,920.898438,606.063947,11


In [ ]:
## ใช้ CTGan เพิ่มข้อมูลฝั่ง focus เพื่อให้มันเท่ากัน

In [17]:
!pip install sdv

  Using cached sdv-1.20.0-py3-none-any.whl.metadata (14 kB)
Using cached sdv-1.20.0-py3-none-any.whl (157 kB)


In [22]:
from sdv.lite import SingleTablePreset
from sdv.metadata import SingleTableMetadata

In [24]:
metadata = SingleTableMetadata()
metadata.detect_from_dataframe(data=df_focus)

In [25]:
synthesizer = SingleTablePreset(metadata, name='FAST_ML')
synthesizer.fit(df_focus)
synthetic_focus = synthesizer.sample(456)

/opt/homebrew/lib/python3.11/site-packages/sdv/lite/single_table.py:52: FutureWarning: The 'SingleTablePreset' is deprecated. For equivalent Fast ML functionality, please use the 'GaussianCopulaSynthesizer'.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/opt/homebrew/lib/python3.11/site-packages/sdv/lite/single_table.py:61: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(META_DEPRECATION_MSG, FutureWarning)
/opt/homebrew/lib/python3.11/site-packages/sdv/single_table/base.py:126: FutureWarning: The 'SingleTableMetadata' is deprecated. Please use the new 'Metadata' class for synthesizers.
  warnings.warn(DEPRECATION_MSG, FutureWarning)
/opt/homebrew/lib/python3.11/site-packages/sdv/single_table/base.py:105: UserWarning: We strongly recommend saving the metadata using 'save_to_json' for replicability in future SDV versions.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/sdv/lite/single_table.py:10

In [29]:
synthetic_focus

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,HR,IBI,HRV,subject
0,32.742537,-12.712715,5.858963,63.457304,2.926168,94.591348,688.953488,594.811197,7
1,44.475001,20.359070,24.632343,63.452062,6.424395,59.997193,1325.106648,675.970607,3
2,17.444562,-59.378125,31.240445,63.429218,6.535110,70.489608,1083.569903,855.872807,10
3,-52.811553,-27.960584,-15.550396,64.560859,3.369857,69.285240,1050.436864,639.560535,14
4,18.945434,-31.877292,-33.615065,64.426941,4.369717,64.245435,1210.502460,568.429220,13
...,...,...,...,...,...,...,...,...,...
451,-19.412742,24.348759,21.816722,64.290345,6.057769,67.731782,937.015094,254.770410,16
452,2.827750,24.391430,-13.170546,64.600525,1.548894,65.451710,1175.527525,609.837556,9
453,-3.425143,-2.263883,-28.134916,64.201297,2.257670,61.574674,1205.546630,679.885862,2
454,-2.858106,-11.138276,-20.309654,64.460857,7.750384,78.422023,1010.821665,977.858219,11


In [30]:
synthetic_focus.insert(8, "label", 1)

In [31]:
synthetic_focus

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,HR,IBI,HRV,label,subject
0,32.742537,-12.712715,5.858963,63.457304,2.926168,94.591348,688.953488,594.811197,1,7
1,44.475001,20.359070,24.632343,63.452062,6.424395,59.997193,1325.106648,675.970607,1,3
2,17.444562,-59.378125,31.240445,63.429218,6.535110,70.489608,1083.569903,855.872807,1,10
3,-52.811553,-27.960584,-15.550396,64.560859,3.369857,69.285240,1050.436864,639.560535,1,14
4,18.945434,-31.877292,-33.615065,64.426941,4.369717,64.245435,1210.502460,568.429220,1,13
...,...,...,...,...,...,...,...,...,...,...
451,-19.412742,24.348759,21.816722,64.290345,6.057769,67.731782,937.015094,254.770410,1,16
452,2.827750,24.391430,-13.170546,64.600525,1.548894,65.451710,1175.527525,609.837556,1,9
453,-3.425143,-2.263883,-28.134916,64.201297,2.257670,61.574674,1205.546630,679.885862,1,2
454,-2.858106,-11.138276,-20.309654,64.460857,7.750384,78.422023,1010.821665,977.858219,1,11


In [32]:
df_balanced = pd.concat([df_all, synthetic_focus], ignore_index=True)
df_balanced

,ACC_x_mean,ACC_y_mean,ACC_z_mean,net_acc_mean,net_acc_std,HR,IBI,HRV,label,subject
0,-46.582292,-6.051042,32.276042,64.614681,4.654549,69.224135,1152.500000,619.704125,0,4
1,-43.976042,-31.740625,-19.667708,64.940738,4.375065,79.507543,915.322581,738.055935,0,4
2,-56.694792,-7.480208,-23.934375,65.363949,1.140627,72.979079,1045.138889,753.696819,0,4
3,-33.206250,-8.705208,-54.866667,64.771843,0.621698,78.275497,1008.620690,855.633087,0,4
4,-33.027083,-8.196875,-54.929167,64.693208,0.778812,68.677229,1160.156250,854.699485,0,4
...,...,...,...,...,...,...,...,...,...,...
1601,-19.412742,24.348759,21.816722,64.290345,6.057769,67.731782,937.015094,254.770410,1,16
1602,2.827750,24.391430,-13.170546,64.600525,1.548894,65.451710,1175.527525,609.837556,1,9
1603,-3.425143,-2.263883,-28.134916,64.201297,2.257670,61.574674,1205.546630,679.885862,1,2
1604,-2.858106,-11.138276,-20.309654,64.460857,7.750384,78.422023,1010.821665,977.858219,1,11


In [43]:
df_balanced.value_counts('label')

label
0    803
1    803
Name: count, dtype: int64

In [35]:
df_all.value_counts('subject')

subject
11    79
10    78
13    78
14    78
15    78
17    78
8     77
3     76
5     76
6     76
7     76
16    76
4     75
9     75
2     74
Name: count, dtype: int64

## Train ML model (xgboost)

In [36]:
from sklearn.model_selection import train_test_split
from xgboost import XGBClassifier
from sklearn.metrics import classification_report

In [38]:
X = df_balanced.drop(columns=["label", "subject"])
y = df_balanced["label"]

In [39]:
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, stratify=y, random_state=42
)

In [40]:
model = XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42)
model.fit(X_train, y_train)

[16:13:28] WARNING: /Users/runner/work/xgboost/xgboost/src/learner.cc:738: 
Parameters: { "use_label_encoder" } are not used.



XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, feature_weights=None, gamma=None,
              grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=None, n_jobs=None,
              num_parallel_tree=None, ...)

In [41]:
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.86      0.87      0.86       161
           1       0.87      0.86      0.86       161

    accuracy                           0.86       322
   macro avg       0.86      0.86      0.86       322
weighted avg       0.86      0.86      0.86       322



In [42]:
# Feature Importances
sorted(zip(X.columns, model.feature_importances_), key=lambda x: x[1], reverse=True)

[('net_acc_std', np.float32(0.30075598)),
 ('ACC_z_mean', np.float32(0.16081502)),
 ('net_acc_mean', np.float32(0.112528905)),
 ('ACC_x_mean', np.float32(0.11215103)),
 ('ACC_y_mean', np.float32(0.10431403)),
 ('IBI', np.float32(0.09809811)),
 ('HRV', np.float32(0.06003404)),
 ('HR', np.float32(0.051302847))]